# Homework 1: Preprocessing and Text Classification

Student Name: Ge Yang

Student ID: 925156

# General Info

<b>Due date</b>: Sunday, 5 Apr 2020 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day (both week and weekend days counted)

<b>Marks</b>: 10% of mark for class (with 9% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/17601/pages/using-jupyter-notebook-and-python?module_item_id=1678430) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

# Overview

In this homework, you'll be working with a collection tweets. The task is to classify whether a tweet constitutes a rumour event. This homework involves writing code to preprocess data and perform text classification.

# 1. Preprocessing (5 marks)

**Instructions**: Run the code below to download the tweet corpus for the assignment. Note: the download may take some time. **No implementation is needed.**

In [2]:
import requests
import os
from pathlib import Path

fname = 'rumour-data.tgz'
data_dir = os.path.splitext(fname)[0] #'rumour-data'

my_file = Path(fname)
if not my_file.is_file():
    url = "https://github.com/jhlau/jhlau.github.io/blob/master/files/rumour-data.tgz?raw=true"
    r = requests.get(url)

    #Save to the current directory
    with open(fname, 'wb') as f:
        f.write(r.content)
        
print("Done. File downloaded:", my_file)


Done. File downloaded: rumour-data.tgz


**Instructions**: Run the code to extract the zip file. Note: the extraction may take a minute or two. **No implementation is needed.**

In [3]:
import tarfile

#decompress rumour-data.tgz
tar = tarfile.open(fname, "r:gz")
tar.extractall()
tar.close()

#remove superfluous files (e.g. .DS_store)
extra_files = []
for r, d, f in os.walk(data_dir):
    for file in f:
        if (file.startswith(".")):
            extra_files.append(os.path.join(r, file))
for f in extra_files:
    os.remove(f)

print("Extraction done.")

Extraction done.


### Question 1 (1.0 mark)

**Instructions**: The corpus data is in the *rumour-data* folder. It contains 2 sub-folders: *non-rumours* and *rumours*. As the names suggest, *rumours* contains all rumour-propagating tweets, while *non-rumours* has normal tweets. Within  *rumours* and *non-rumours*, you'll find some sub-folders, each named with an ID. Each of these IDs constitutes an 'event', where an event is defined as consisting a **source tweet** and its **reactions**.

An illustration of the folder structure is given below:

    rumour-data
        - rumours
            - 498254340310966273
                - reactions
                    - 498254340310966273.json
                    - 498260814487642112.json
                - source-tweet
                    - 498254340310966273.json
        - non-rumours

Now we need to gather the tweet messages for rumours and non-rumour events. As the individual tweets are stored in json format, we need to use a json parser to parse and collect the actual tweet message. The function `get_tweet_text_from_json(file_path)` is provided to do that.

**Task**: Complete the `get_events(event_dir)` function. The function should return **a list of events** for a particular class of tweets (e.g. rumours), and each event should contain the source tweet message and all reaction tweet messages.

**Check**: Use the assertion statements in *"For your testing"* below for the expected output.

In [4]:
import json

def get_tweet_text_from_json(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
        return data["text"]
    
def get_events(event_dir):
    event_list = []
    for event in sorted(os.listdir(event_dir)):
        ###
        # Your answer BEGINS
        ###
        
        src_rec_list = []
        
        # Extract source text
        source_path = os.path.join(event_dir, event, 'source-tweet', event + '.json')
        src_txt = get_tweet_text_from_json(source_path)
        src_rec_list.append(src_txt)
        
        # Extract reacions texts
        reactions_path = os.path.join(event_dir, event, 'reactions')
        reactions_list = os.listdir(reactions_path)
    
        for reaction in sorted(reactions_list):
            reaction_txt = get_tweet_text_from_json(os.path.join(reactions_path, reaction))
            src_rec_list.append(reaction_txt)
        
        # Store in the list
        event_list.append(src_rec_list)
        ###
        # Your answer ENDS HERE
        ###
        
    return event_list
    
#a list of events, and each event is a list of tweets (source tweet + reactions)    
rumour_events = get_events(os.path.join(data_dir, "rumours"))
nonrumour_events = get_events(os.path.join(data_dir, "non-rumours"))

print("Number of rumour events =", len(rumour_events))
print("Number of non-rumour events =", len(nonrumour_events))

Number of rumour events = 500
Number of non-rumour events = 1000


**For your testing:**

In [5]:
assert(len(rumour_events) == 500)
assert(len(nonrumour_events) == 1000)

### Question 2 (1.0 mark)

**Instructions**: Next we need to preprocess the collected tweets to create a bag-of-words representation. The preprocessing steps required here are: (1) tokenize each tweet into individual word tokens (using NLTK `TweetTokenizer`); and (2) remove stopwords (based on NLTK `stopwords`).

**Task**: Complete the `preprocess_events(event)` function. The function takes **a list of events** as input, and returns **a list of preprocessed events**. Each preprocessed event should have a dictionary of words and frequencies.

**Check**: Use the assertion statements in *"For your testing"* below for the expected output.

In [6]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from collections import defaultdict

tt = TweetTokenizer()
stopwords = set(stopwords.words('english'))

def preprocess_events(events):
    ###
    # Your answer BEGINS HERE
    ###
    preprocess_events = []
    for event in events:
        event_map = defaultdict(int)
        for tweet in event:
            tokenized_tweet = tt.tokenize(tweet)
            tokens_filtered_sw = [word for word in tokenized_tweet if not word.lower() in stopwords]
            for word in tokens_filtered_sw:
                event_map[word] += 1
        
        preprocess_events.append(event_map)
        
    return preprocess_events
    ###
    # Your answer ENDS HERE
    ###

preprocessed_rumour_events = preprocess_events(rumour_events)
preprocessed_nonrumour_events = preprocess_events(nonrumour_events)
print(preprocessed_rumour_events)
print("Number of preprocessed rumour events =", len(preprocessed_rumour_events))
print("Number of preprocessed non-rumour events =", len(preprocessed_nonrumour_events))

[defaultdict(<class 'int'>, {'Michael': 2, 'Brown': 2, '17': 4, 'yr': 2, 'old': 4, 'boy': 5, 'shot': 7, '10x': 2, '&': 4, 'killed': 2, 'police': 10, '#Ferguson': 3, 'today': 2, '.': 47, 'Media': 2, 'reports': 2, '"': 22, 'shoot': 2, 'man': 3, '#blackboysonly': 2, '@AmeenaGK': 38, '@jaythenerdkid': 3, 'long': 1, 'conservative': 1, 'pundit': 1, 'finds': 1, 'pic': 1, 'messing': 1, 'around': 2, 'friends': 1, 'ID': 1, '@d_m_elms': 2, "they'll": 1, 'drag': 1, 'entire': 1, 'history': 1, 'every': 2, 'stolen': 1, 'sip': 1, "dad's": 1, 'beer': 1, ',': 17, 'second-hand': 1, 'tendril': 1, 'pot': 1, 'smoke': 1, 'watch': 1, 'traces': 1, 'back': 6, 'bogus': 1, 'allegations': 1, 'something': 1, 'kindergarten': 1, 'justify': 1, 'dangerous': 1, 'CHILD': 1, 'KILLED': 1, 'PROTECT': 1, 'CANDY': 1, 'headline': 1, '@CyMadD0x': 6, 'unarmed': 1, 'black': 8, 'year': 2, 'cops': 2, 'instead': 1, 'fellow': 1, 'blacks': 1, '...': 2, 'Better': 1, 'make': 2, 'national': 1, 'news': 1, 'know': 2, 'right': 2, '?': 15, '

**For your testing**:

In [6]:
assert(len(preprocessed_rumour_events) == 500)
assert(len(preprocessed_nonrumour_events) == 1000)

**Instructions**: Hashtags (i.e. topic tags which start with #) pose an interesting tokenisation problem because they often include multiple words written without spaces or capitalization. Run the code below to collect all unique hashtags in the preprocessed data. **No implementation is needed.**



In [7]:
def get_all_hashtags(events):
    hashtags = set([])
    for event in events:
        for word, frequency in event.items():
            if word.startswith("#"):
                hashtags.add(word)
    return hashtags

hashtags = get_all_hashtags(preprocessed_rumour_events + preprocessed_nonrumour_events)
print("Number of hashtags =", len(hashtags))

Number of hashtags = 1829


### Question 3 (2.0 mark)

**Instructions**: Our task here to tokenize the hashtags, by implementing a reversed version of the MaxMatch algorithm discussed in class, where matching begins at the end of the hashtag and progresses backwards. NLTK has a list of words that you can use for matching, see starter code below. Be careful about efficiency with respect to doing word lookups. One extra challenge you have to deal with is that the provided list of words includes only lemmas: your MaxMatch algorithm should match inflected forms by converting them into lemmas using the NLTK lemmatizer before matching. When lemmatising a word, you also need to provide the part-of-speech tag of the word. You should use `nltk.tag.pos_tag` for doing part-of-speech tagging.

Note that the list of words is incomplete, and, if you are unable to make any longer match, your code should default to matching a single letter. Create a new list of tokenized hashtags (this should be a list of lists of strings) and use slicing to print out the last 20 hashtags in the list.

For example, given "#speakup", the algorithm should produce: \["#", "speak", "up"\]. And note that you do not need to delete the hashtag symbol ("#") from the tokenised outputs.

**Task**: Complete the `tokenize_hashtags(hashtags)` function by implementing a reversed MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of word tokens".

**Check**: Use the assertion statements in *"For your testing"* below for the expected output.

In [8]:
from nltk.corpus import wordnet

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
words = set(nltk.corpus.words.words()) #a list of words provided by NLTK

def tokenize_hashtags(hashtags):
    ###
    # Your answer BEGINS HERE
    ###
    hash_map = {}
    
    for hashtag in hashtags:
        tokens = ReverseMaxMatch(hashtag, words)
        hash_map[hashtag] = tokens
        
    return hash_map

# add pos tagging
def ReverseMaxMatch(s, dic):
    index = len(s)
    res = []
    while index > 0:
        word = None
        for i in range(len(s), 0, -1):
            if index - i < 0:
                continue
            else:
                # choose the longest substring and shrink the length
                pieces = s[index-i: index]
                lemma = lemmatizer.lemmatize(pieces, get_wordnet_pos(pieces)).lower()
                
                # deal with the word with a Capitol inital in the dictionary.
                Bf_lemma = lemma[0].upper() + lemma[1:]
                
                # deal with the word all upper case in the dictionary
                Big_lemma = lemma.upper()
                if lemma in dic or Big_lemma in dic or Bf_lemma in dic:
                    word = pieces
                    res.append(word)
                    index -= i
                    break
            
        if word is None:
            res.append(s[index-1])
            index -= 1

    return res[::-1]

def get_wordnet_pos(word):
    # Map POS tag to first character lemmatize() accepts
    # from https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
    # rewrite by Ge Yang
    tag = nltk.tag.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

    ###
    # Your answer ENDS HERE
    ###

tokenized_hashtags = tokenize_hashtags(hashtags)
print(list(tokenized_hashtags.items())[:20])

[('#kmov', ['#', 'k', 'mo', 'v']), ('#Fair', ['#', 'Fair']), ('#watchyourback', ['#', 'watch', 'your', 'back']), ('#moron', ['#', 'moron']), ('#usausausa', ['#', 'u', 'sa', 'u', 'sa', 'u', 'sa']), ('#StLouisCountyPD', ['#', 'St', 'Louis', 'County', 'P', 'D']), ('#FromMyColdDeadHands', ['#', 'From', 'My', 'Cold', 'D', 'ea', 'dHan', 'ds']), ('#LoudAndProud', ['#', 'Lou', 'dAnd', 'P', 'roud']), ('#YPG', ['#', 'Y', 'P', 'G']), ('#khilafah', ['#', 'khila', 'f', 'ah']), ('#Liberal', ['#', 'Liberal']), ('#Islam', ['#', 'Islam']), ('#LindtCafeSiege', ['#', 'L', 'ind', 't', 'Ca', 'fe', 'Siege']), ('#tpp', ['#', 't', 'p', 'p']), ('#GunControl', ['#', 'G', 'unControl']), ('#BeSafeOttawa', ['#', 'Be', 'Safe', 'Ottawa']), ('#LiftTheBan', ['#', 'Lift', 'TheBan']), ('#WonderWoman', ['#', 'Wonder', 'Woman']), ('#wow', ['#', 'wow']), ('#diversity', ['#', 'diversity'])]


**For your testing:**

In [9]:
assert(len(tokenized_hashtags) == len(hashtags))

### Question 4 (1.0 mark)

**Instructions**: Now that we have the tokenized hashtags, we need to go back and update the bag-of-words representation for each event.

**Task**: Complete the ``update_event_bow(events)`` function. The function takes **a list of preprocessed events**, and for each event, it looks for every hashtag it has and updates the bag-of-words dictionary with the tokenized hashtag tokens. Note: you do not need to delete the counts of the original hashtags when updating the bag-of-words (e.g., if a document has "#speakup":2 in its bag-of-words representation, you do not need to delete this hashtag and its counts).

In [10]:
import os
def update_event_bow(events):
    ###
    # Your answer BEGINS HERE
    ###
    
    for tags, tokens in tokenized_hashtags.items():
        for event in events:
            if tags in event:
                for token in tokens:
                    event[token[0]] += 1
    
    ###
    # Your answer ENDS HERE
    ###
            
update_event_bow(preprocessed_rumour_events)
update_event_bow(preprocessed_nonrumour_events)

print("Number of preprocessed rumour events =", len(preprocessed_rumour_events))
print("Number of preprocessed non-rumour events =", len(preprocessed_nonrumour_events))

Number of preprocessed rumour events = 500
Number of preprocessed non-rumour events = 1000


# Text Classification (4 marks)

### Question 5 (1.0 mark)

**Instructions**: Here we are interested to do text classification, to predict, given a tweet and its reactions, whether it is a rumour or not. The task here is to create training, development and test partitions from the preprocessed events and convert the bag-of-words representation into feature vectors.

**Task**: Using scikit-learn, create training, development and test partitions with a 60%/20%/20% ratio. Remember to preserve the ratio of rumour/non-rumour events for all your partitions. Next, turn the bag-of-words dictionary of each event into a feature vector, using scikit-learn `DictVectorizer`.

In [11]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
import numpy as np

vectorizer = DictVectorizer()

###
# Your answer BEGINS HERE
###

def get_classification(rm_length, nr_length):
    classification = []
    for i in range(rm_length):
        classification.append('rumor')
    for j in range(nr_length):
        classification.append('nonrumor')
    return classification

# Divid the events into train, dev and test set following the ratio
rm_tr, rm_dev_te = train_test_split(preprocessed_rumour_events, test_size=0.4)
rm_dev, rm_te = train_test_split(rm_dev_te, test_size=0.5)

nr_tr, nr_dev_te = train_test_split(preprocessed_nonrumour_events, test_size=0.4)
nr_dev, nr_te = train_test_split(nr_dev_te, test_size=0.5)

# merge the rumor set with non_rumor set
train_set = rm_tr + nr_tr
dev_set = rm_dev + nr_dev
test_set = rm_te + nr_te

# get classifications for each set
train_classifications = get_classification(len(rm_tr), len(nr_tr))
dev_classifications = get_classification(len(rm_dev), len(nr_dev))
test_classifications = get_classification(len(rm_te), len(nr_te))

# vectorize the data set
vectorizer = DictVectorizer()
train_data = vectorizer.fit_transform(train_set)
dev_data = vectorizer.transform(dev_set)
test_data = vectorizer.transform(test_set)

###
# Your answer ENDS HERE
###

print("Vocabulary size =", len(vectorizer.vocabulary_))

Vocabulary size = 26520


### Question 6 (2.0 mark)

**Instructions**: Now, let's build some classifiers. Here, we'll be comparing Naive Bayes and Logistic Regression. For each, you need to first find a good value for their main regularisation (hyper)parameters, which you should identify using the scikit-learn docs or other resources. Use the development set you created for this tuning process; do **not** use cross-validation in the training set, or involve the test set in any way. You don't need to show all your work, but you do need to print out the accuracy with enough different settings to strongly suggest you have found an optimal or near-optimal choice. We should not need to look at your code to interpret the output.

**Task**: Implement two text classifiers: Naive Bayes and Logistic Regression. Tune the hyper-parameters of these classifiers and print the task performance for different hyper-parameter settings.

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

###
# Your answer BEGINS HERE
###

# check range of parameter, check 

best_accuracy = 0

# tune the smooth parameter alpha
alps = np.arange(1, 0, -0.1)
best_alpha = 0

for alp in alps:
    mnb = MultinomialNB(alp)
    mnb = mnb.fit(train_data, train_classifications)
    accuracy = accuracy_score(mnb.predict(dev_data), dev_classifications)
    
    # look for highest accuracy
    if best_accuracy < accuracy:
        best_accuracy = accuracy
        best_alpha = alp
    
    print('alpha:{:.3f}, Accuracy:{:.3f}'.format(alp, accuracy))

print('For Naive Bayes:')
print('best_accuracy is {:.3f}, with alpha at {:.3f}'.format(best_accuracy, best_alpha))

best_accuracy = 0

# tune parameter C
C_param_range = np.arange(1, 0, -0.1)
best_c = 0

for c in C_param_range:
    lgr = LogisticRegression(C=c)
    lgr = lgr.fit(train_data, train_classifications)
    accuracy = accuracy_score(lgr.predict(dev_data), dev_classifications)
    
    # look for highest accuracy
    if best_accuracy < accuracy:
        best_accuracy = accuracy
        best_c = c
    
    print('C:{:.3f}, Accuracy:{:.3f}'.format(c, accuracy))

print('For Logistric Regression:')
print('best_accuracy is {:.3f}, with c at {:.3f}'.format(best_accuracy, best_c))

###
# Your answer ENDS HERE
###

alpha:1.000, Accuracy:0.810
alpha:0.900, Accuracy:0.807
alpha:0.800, Accuracy:0.813
alpha:0.700, Accuracy:0.820
alpha:0.600, Accuracy:0.813
alpha:0.500, Accuracy:0.810
alpha:0.400, Accuracy:0.810
alpha:0.300, Accuracy:0.807
alpha:0.200, Accuracy:0.800
alpha:0.100, Accuracy:0.803
For Naive Bayes:
best_accuracy is 0.820, with alpha at 0.700
C:1.000, Accuracy:0.810
C:0.900, Accuracy:0.810
C:0.800, Accuracy:0.810


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


C:0.700, Accuracy:0.810
C:0.600, Accuracy:0.813
C:0.500, Accuracy:0.803
C:0.400, Accuracy:0.803
C:0.300, Accuracy:0.807
C:0.200, Accuracy:0.810
C:0.100, Accuracy:0.810
For Logistric Regression:
best_accuracy is 0.813, with c at 0.600


### Question 7 (1.0 mark)

**Instructions**: Using the best settings you have found, compare the two classifiers based on performance in the test set. Print out both accuracy and macro-averaged F-score for each classifier. Be sure to label your output.

**Task**: Compute test performance in terms of accuracy and macro-averaged F-score for both Naive Bayes and Logistic Regression, using optimal hyper-parameter settings.

In [45]:
###
# Your answer BEGINS HERE
###

from sklearn.metrics import f1_score

# use best hyperparameter
clfs = [MultinomialNB(alpha=best_alpha),LogisticRegression(C=best_c)]

# simple validation on the accuracy and macro f score
def do_simple_validation(clfs, train_data, train_classifications, test_data, test_classifications):
    print('simple validation')
    for clf in clfs:
        clf = clf.fit(train_data, train_classifications)
        predictions = clf.predict(test_data)
        print(str(clf)[:12])
        print ("accuracy: {:3f}".format(accuracy_score(test_classifications,predictions)))
        print ("macro_f_score: {:3f}".format(f1_score(test_classifications, predictions, average='macro')))

# 10 folds cross validation and full report
def do_multiple_10foldcrossvalidation(clfs,data,classifications):
    print('10 fold cross validation')
    for clf in clfs:
        predictions = model_selection.cross_val_predict(clf, data,classifications, cv=10)
        print(str(clf)[:12])
        print ("accuracy")
        print (accuracy_score(classifications,predictions))
        print (classification_report(classifications,predictions))

do_simple_validation(clfs, train_data, train_classifications, test_data, test_classifications)
do_multiple_10foldcrossvalidation(clfs,test_data,test_classifications)
    
###
# Your answer ENDS HERE
###

simple validation
MultinomialN
accuracy: 0.776667
macro_f_score: 0.746823
LogisticRegr
accuracy: 0.813333
macro_f_score: 0.773353
10 fold cross validation
MultinomialN
accuracy
0.7933333333333333
              precision    recall  f1-score   support

    nonrumor       0.78      0.96      0.86       200
       rumor       0.85      0.46      0.60       100

   micro avg       0.79      0.79      0.79       300
   macro avg       0.82      0.71      0.73       300
weighted avg       0.80      0.79      0.77       300



/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegr
accuracy
0.7966666666666666
              precision    recall  f1-score   support

    nonrumor       0.80      0.92      0.86       200
       rumor       0.77      0.55      0.64       100

   micro avg       0.80      0.80      0.80       300
   macro avg       0.79      0.74      0.75       300
weighted avg       0.79      0.80      0.79       300

